In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

import pandas as pd
from pandas.io.json import json_normalize
import altair as alt

In [ ]:
filename = "../data/nadmiral_parcel_info.geojson"
json_data = {}
with open(filename) as data_file:    
    json_data = json.load(data_file)
print(len(json_data['features']))

In [ ]:
all_filename = "../data/point_info.json"
point_data = {}
with open(all_filename) as data_file:    
    point_data = json.load(data_file)
print(len(point_data))

In [ ]:
print(json_data['features'][0])

In [ ]:
features = [p['properties'] for p in json_data['features']]

In [ ]:
all_df = pd.DataFrame(features)
print(len(all_df))
all_df.head()

In [ ]:
point_df = pd.DataFrame([p['attributes'] for p in point_data])
point_df.head()

In [ ]:
all_df = all_df.merge(point_df, how="left", on="OBJECTID", suffixes=["", "_all"])
print(len(all_df))

In [ ]:
all_df['APPVALUE'] = pd.to_numeric(all_df['APPVALUE'])
all_df['LOTSQFT'] = pd.to_numeric(all_df['LOTSQFT'])

In [ ]:
all_df.dtypes

In [ ]:
alt.Chart(all_df).mark_bar().encode(
    x=alt.X('PRESENTUSE'),
    y='count()'
)

In [ ]:
sfam_df = all_df[all_df['PRESENTUSE'] == 'Single Family(Res Use/Zone)']

In [ ]:
alt.Chart(sfam_df).mark_point().encode(
    x=alt.X('LOTSQFT:Q'),
    y=alt.Y('APPVALUE:Q'),
    tooltip=["PIN", "ADDR_FULL", 'TAXPAYERNAME']
).interactive()

In [ ]:


by_street = sfam_df.groupby('ADDR_SN').agg({'APPVALUE': ['mean', 'max', 'min', 'count']}).reset_index().sort_values(by=('APPVALUE', 'mean'), ascending=False)

by_street.columns = [' '.join(col).strip() for col in by_street.columns.values]

alt.Chart(by_street).mark_bar().encode(
    alt.X("APPVALUE mean:Q"),
    y=alt.Y('ADDR_SN'),
)

In [ ]:
by_street